# **Psudocode for creating a decision tree**
1. create a new node.
2. checks if the node meets any stopping criteria:
    - purity
    - max depth
    - minimum information gain
4. If a stopping criterion is met, return the node as leaf node and stops further splitting.
5. If no stopping criterion is met, calculates information gain for all features at the current node and selectes the highest.
6. splits the datasets at the current node using the selected feature by creating two nodes: left and right
7. repeat step 2 to 6 for each new node created after the split. 

# **1. Visualize dataset**


In [260]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [261]:
cancer = pd.read_csv('datasets/Liver_GSE14520_U133A.csv')
cancer.head(10000)         

,samples,type,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,...,AFFX-r2-Ec-bioD-3_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-3_at,AFFX-r2-P1-cre-5_at,AFFX-ThrX-3_at,AFFX-ThrX-5_at,AFFX-ThrX-M_at,AFFX-TrpnX-3_at,AFFX-TrpnX-5_at,AFFX-TrpnX-M_at
0,GSM362958.CEL.gz,HCC,6.801198,4.553189,6.787790,5.430893,3.250222,6.272688,3.413405,3.374910,...,10.735084,10.398843,12.298551,12.270505,3.855588,3.148321,3.366087,3.199008,3.160388,3.366417
1,GSM362959.CEL.gz,HCC,7.585956,4.193540,3.763183,6.003593,3.309387,6.291927,3.754777,3.587603,...,11.528447,11.369919,12.867048,12.560433,4.016561,3.282867,3.541994,3.548680,3.460083,3.423348
2,GSM362960.CEL.gz,HCC,7.803370,4.134075,3.433113,5.395057,3.476944,5.825713,3.505036,3.687333,...,10.892460,10.416151,12.356337,11.888482,3.839367,3.598851,3.516791,3.484089,3.282626,3.512024
3,GSM362964.CEL.gz,HCC,6.920840,4.000651,3.754500,5.645297,3.387530,6.470458,3.629249,3.577534,...,10.686871,10.524836,12.006596,11.846195,3.867602,3.180472,3.309547,3.425501,3.166613,3.377499
4,GSM362965.CEL.gz,HCC,6.556480,4.599010,4.066155,6.344537,3.372081,5.439280,3.762213,3.440714,...,11.014454,10.775566,12.657182,12.573076,4.091440,3.306729,3.493704,3.205771,3.378567,3.392938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,GSM712537.CEL.gz,normal,6.857025,3.965635,3.883590,6.190071,3.054398,6.339694,3.993988,3.566907,...,11.354129,11.039429,12.779742,12.677145,3.883899,3.362722,3.550832,3.142141,3.154087,3.412942
353,GSM712538.CEL.gz,normal,7.291103,3.779197,4.024521,5.456527,3.275127,5.870166,3.872936,3.729544,...,11.343014,11.124686,12.861682,12.691833,3.756814,3.198867,3.359730,3.020554,3.250578,3.189661
354,GSM712539.CEL.gz,normal,7.493958,4.200727,3.791528,5.576578,3.118160,5.400790,3.927906,3.421077,...,11.542476,11.371326,13.228540,12.999379,5.796103,3.424177,4.173774,3.205583,3.068978,3.148050
355,GSM712540.CEL.gz,normal,6.835236,4.112805,3.907909,5.786022,3.164424,5.996150,4.019206,3.533469,...,11.855368,11.659300,13.176290,13.018847,6.537310,4.140005,4.578915,3.185101,3.086303,3.224059


In [262]:
cancer.shape

(357, 22279)

In [263]:
cancer['type'].value_counts()

type
HCC       181
normal    176
Name: count, dtype: int64

In [264]:
from sklearn.model_selection import train_test_split
X = cancer.drop(columns =['type','samples'],axis = 1)
y = cancer['type']

from sklearn.preprocessing import LabelEncoder
y_encoded = LabelEncoder().fit_transform(y)
X_train,X_test,y_train,y_test = train_test_split(X ,y_encoded,test_size = 0.3,stratify = y, random_state = 42)

print(X_train.shape)
print(y_train.shape)
print('Number of training exampes:', len(X_train))

(249, 22277)
(249,)
Number of training exampes: 249


# **2. Train a two level stacked ensamble model**

The SVM acts as a base (level-0) model

In [265]:
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
# train svm and get pred on X_train
svm = svm.SVC(class_weight = 'balanced', random_state = 42)
svm.fit(X_train, y_train)
svm_train_pred = cross_val_predict(svm, X_train, y_train, cv = 5)
print(svm_train_pred)
print(len(svm_train_pred))

[0 0 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 1 0 0 1 1 1 0 0 0 1 1 0 0
 1 0 1 1 0 1 1 1 0 0 0 0 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 0 0 0 0 0 1 1 1 0
 1 1 0 0 0 1 0 0 1 1 0 1 0 0 1 1 1 0 1 1 0 0 1 1 0 0 0 1 1 0 1 0 0 1 1 0 1
 1 0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 0 0 0 1 0 1 1 1 1 1 1 0 1 1 0 0 0 0 1 1 1
 1 0 1 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 0 0 1 1 0 1 0 1 1 1 0
 0 0 1 1 0 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 1
 0 1 1 1 0 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 1 0 0 1 1 0 0]
249


In [266]:
pred_svm = svm.predict(X_test)
# Each sample has original feature and added one new feature
X_train_meta = np.column_stack((X_train, svm_train_pred))
X_test_meta = np.column_stack((X_test, pred_svm))
# after add one feature from SVM output
print(X_train_meta.shape)
print(X_test_meta.shape)

(249, 22278)
(108, 22278)


The Decision Tree is the meta (level-1) model that learns from both the original features and the SVM's predictions.

In [267]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

dt = DecisionTreeClassifier(criterion = 'gini', max_depth = 3, splitter = 'best', random_state = 42)
dt.fit(X_train_meta, y_train)

dt_pred = dt.predict(X_test_meta)
print("f1_score:", f1_score(y_test, dt_pred))

cv_scores = cross_val_score(dt, X_train_meta, y_train, cv=5)
print("Cross val scores:", cv_scores)
print("Mean cross val score:", cv_scores.mean())

f1_score: 0.918918918918919
Cross val scores: [0.84       0.92       0.94       0.96       0.89795918]
Mean cross val score: 0.9115918367346939


# **3. perform feature selection using Recursive Feature Selection**

In [268]:
from sklearn.feature_selection import RFE

dt_f1_score =[]
dt_cv_score=[]
n_feature_list = [25,50,75,100,125,150,175,200,225,250,275,300,325,350,375,400,425,450,500]
for k in n_feature_list:
  dt = DecisionTreeClassifier(criterion = 'gini', max_depth = 3, splitter = 'best', random_state = 42)
  RFE_selector = RFE(estimator=dt, n_features_to_select=k, step=0.1)
  RFE_selector.fit(X_train_meta, y_train)
  sel_X_train = RFE_selector.transform(X_train_meta)
  sel_X_test = RFE_selector.transform(X_test_meta)

  dt.fit(sel_X_train, y_train)
  dt_preds = dt.predict(sel_X_test)
  dt_f1_score.append(f1_score(y_test, dt_preds, average='weighted'))

  #add cross validation scores
  dt_cv = cross_val_score(dt, sel_X_train, y_train, cv=5, scoring='f1_weighted')
  dt_cv_score.append(round(dt_cv.mean(), 3))

print(dt_f1_score)

best_index = dt_f1_score.index(max(dt_f1_score))
best_k = n_feature_list[best_index]
best_score_dt = dt_f1_score[best_index]

print(f"Best F1 score: {best_score_dt:.4f} with {best_k} features")
print("cross validation score:", np.mean(dt_cv_score))

[0.9072166259666259, 0.9165594165594165, 0.9165594165594165, 0.9165594165594165, 0.9072166259666259, 0.9165594165594165, 0.9072166259666259, 0.9165594165594165, 0.8789767382535355, 0.8789767382535355, 0.9072166259666259, 0.9072166259666259, 0.9165594165594165, 0.9165594165594165, 0.9165594165594165, 0.9165594165594165, 0.9165594165594165, 0.9165594165594165, 0.9165594165594165]
Best F1 score: 0.9166 with 50 features
cross validation score: 0.9635789473684211


In [269]:
original_feature_names = X.columns.tolist()  
meta_feature_name = 'svm_prediction'
all_feature_names = original_feature_names + [meta_feature_name] 

# View the selected features

In [270]:
RFE_selector = RFE(estimator=dt, n_features_to_select = 50, step = 50)
RFE_selector.fit(X_train_meta, y_train)
selected_feature_mask = RFE_selector.get_support()
selected_feature_indices = np.where(selected_feature_mask)[0]
selected_feature_names = [all_feature_names[i] for i in selected_feature_indices]

print(f"Selected top {len(selected_feature_names)} features:")
print(selected_feature_names)

Selected top 50 features:
['200917_s_at', '200918_s_at', '200919_at', '200920_s_at', '200921_s_at', '200922_at', '200923_at', '200924_s_at', '200925_at', '200926_at', '200927_s_at', '200928_s_at', '200929_at', '200930_s_at', '200931_s_at', '200932_s_at', '200933_x_at', '200934_at', '200935_at', '200936_at', '200937_s_at', '200938_s_at', '200939_s_at', '200940_s_at', '200941_at', '200942_s_at', '200943_at', '200944_s_at', '200945_s_at', '200946_x_at', '200947_s_at', '200948_at', '200949_x_at', '200950_at', '200951_s_at', '200952_s_at', '200953_s_at', '200954_at', '200957_s_at', '200974_at', '200975_at', '200976_s_at', '200977_s_at', '200978_at', '201143_s_at', '208664_s_at', '209365_s_at', 'AFFX-LysX-5_at', 'AFFX-r2-Bs-lys-M_at', 'svm_prediction']


In [271]:
# Step 1: Subset X_train_meta to only selected columns
sel_X_train_best = X_train_meta[:, selected_feature_indices]

# Step 2: Fit DT on selected features
dt_final = DecisionTreeClassifier(criterion = 'gini', random_state=42, max_depth=3, min_samples_leaf = 2)

dt_final.fit(sel_X_train_best, y_train)

# Step 3: Extract feature importances
importances = dt_final.feature_importances_
importance_df = pd.DataFrame({
    'Feature': selected_feature_names,
    'Importance': importances
})

# Step 4: Sort and print top 10 or 20
importance_df_sorted = importance_df.sort_values(by='Importance', ascending=False)

print("Top 10 most important features:")
print(importance_df_sorted.head(10))

print("\nTop 20 most important features:")
print(importance_df_sorted.head(20))

Top 10 most important features:
        Feature  Importance
46  209365_s_at    0.898400
45  208664_s_at    0.061777
44  201143_s_at    0.031854
38  200957_s_at    0.007969
36  200953_s_at    0.000000
27  200944_s_at    0.000000
28  200945_s_at    0.000000
29  200946_x_at    0.000000
30  200947_s_at    0.000000
31    200948_at    0.000000

Top 20 most important features:
        Feature  Importance
46  209365_s_at    0.898400
45  208664_s_at    0.061777
44  201143_s_at    0.031854
38  200957_s_at    0.007969
36  200953_s_at    0.000000
27  200944_s_at    0.000000
28  200945_s_at    0.000000
29  200946_x_at    0.000000
30  200947_s_at    0.000000
31    200948_at    0.000000
32  200949_x_at    0.000000
33    200950_at    0.000000
34  200951_s_at    0.000000
35  200952_s_at    0.000000
0   200917_s_at    0.000000
37    200954_at    0.000000
1   200918_s_at    0.000000
39    200974_at    0.000000
40    200975_at    0.000000
41  200976_s_at    0.000000


In [273]:
from sklearn.tree import export_text
print(export_text(dt_final, feature_names=selected_feature_names))

|--- 209365_s_at <= 6.54
|   |--- 208664_s_at <= 3.53
|   |   |--- class: 1
|   |--- 208664_s_at >  3.53
|   |   |--- 200957_s_at <= 5.84
|   |   |   |--- class: 0
|   |   |--- 200957_s_at >  5.84
|   |   |   |--- class: 0
|--- 209365_s_at >  6.54
|   |--- 201143_s_at <= 4.76
|   |   |--- class: 0
|   |--- 201143_s_at >  4.76
|   |   |--- class: 1



### Map Probe ID to Gene Symbol in R

Use this R code to look up the gene name from a probe ID (e.g., `1553611_s_at`):

```r
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("hgu133plus2.db")
library(hgu133plus2.db)

select(hgu133plus2.db,
       keys = "1553611_s_at",  # replace with your probe ID
       columns = c("SYMBOL", "GENENAME"),
       keytype = "PROBEID")



Example output: 
      PROBEID       SYMBOL                   GENENAME
1 1553611_s_at      KLHL35 kelch like family member 35
